<a href="https://colab.research.google.com/github/radr44/Iterative_Convolutional_Neural_Network/blob/master/checkpoints/training%26testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import numpy as np
import cv2 as cv
import subsampling
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import ICNN
import preprocessing
import copy

In [0]:
data_transforms = transforms.Compose([
        transforms.Resize((96,144)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor() ])

In [0]:
data_dir = '/content/drive/My Drive/Superset'
image_dataset = datasets.ImageFolder(data_dir,data_transforms)
class_names = image_dataset.classes
train_set, hold_set = torch.utils.data.random_split(image_dataset, [23582,10000])
train_set, throw_set = torch.utils.data.random_split(train_set, [23580,2] )
val_set, test_set = torch.utils.data.random_split(hold_set, [5000,5000])
dataset_train = torch.utils.data.DataLoader(train_set, batch_size = 5, shuffle = True, num_workers = 4)
dataset_val = torch.utils.data.DataLoader(val_set, batch_size = 5, shuffle = True, num_workers = 4)
dataset_test = torch.utils.data.DataLoader(test_set, batch_size = 5, shuffle = True, num_workers = 4)


In [0]:
def train_model_p1(model, criterion, optimizer, scheduler, num_epochs=10):
  #train whole network using the fc layers of last ucnn
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'dev']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for batch_idx,(inputs, labels) in enumerate(dataset_combined[phase]):
               
               
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                if phase== 'train':
                    if batch_idx % 50 == 0:
                        print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                      epoch, batch_idx*len(inputs), len(dataset_combined["train"].dataset), 100*batch_idx / len(dataset_train), loss.data, float(running_corrects*100) / float(5*(batch_idx+1))))
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'dev' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    print(scheduler)

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [0]:
dataset_combined = {"train":dataset_train,"dev":dataset_val}
dataset_sizes = {"train":len(train_set),"dev":len(val_set)}

In [0]:
ucnn_1 = ICNN.micro_cnn_Alexnet(k=0)
ucnn_2 = ICNN.micro_cnn_Alexnet(k=1)
ucnn_3 = ICNN.micro_cnn_Alexnet(k=2)
ucnn_4 = ICNN.micro_cnn_Alexnet(k=3)
ucnn_5 = ICNN.micro_cnn_Alexnet(k=4)
ucnn_6 = ICNN.micro_cnn_Alexnet(k=5)
ucnn_7 = ICNN.micro_cnn_Alexnet(k=6)
ucnn_8 = ICNN.micro_cnn_Alexnet(k=7)
ucnn_9 = ICNN.micro_cnn_Alexnet(k=8)
ucnn_10 = ICNN.micro_cnn_Alexnet(k=9,nfc=512)
ucnn = [ucnn_1,ucnn_2,ucnn_3,ucnn_4,ucnn_5,ucnn_6]

In [0]:
icnn=ICNN.ICNN_Alexnet(ucnn=ucnn,batch_size=5)
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.Adam(icnn.parameters(),lr=0.0001)
lmbda = lambda epoch: 0.95

mul_lr_scheduler = lr_scheduler.MultiplicativeLR(optimizer_ft,lr_lambda=lmbda)

In [0]:
len(list(icnn.parameters()))

96

In [0]:
model_ft = train_model_p1(icnn, criterion, optimizer_ft, mul_lr_scheduler,num_epochs=8)

Epoch 0/7
----------
Epoch : 0 [0/23580 (0%)]	Loss: 2.730670	 Accuracy:0.000%
Epoch : 0 [250/23580 (1%)]	Loss: 2.686708	 Accuracy:7.451%
Epoch : 0 [500/23580 (2%)]	Loss: 2.705427	 Accuracy:9.901%
Epoch : 0 [750/23580 (3%)]	Loss: 2.291590	 Accuracy:12.053%
Epoch : 0 [1000/23580 (4%)]	Loss: 2.184672	 Accuracy:17.015%
Epoch : 0 [1250/23580 (5%)]	Loss: 2.016990	 Accuracy:21.514%
Epoch : 0 [1500/23580 (6%)]	Loss: 1.366412	 Accuracy:25.714%
Epoch : 0 [1750/23580 (7%)]	Loss: 0.384138	 Accuracy:29.744%
Epoch : 0 [2000/23580 (8%)]	Loss: 1.373826	 Accuracy:32.968%
Epoch : 0 [2250/23580 (10%)]	Loss: 1.090856	 Accuracy:36.098%
Epoch : 0 [2500/23580 (11%)]	Loss: 1.014690	 Accuracy:39.521%
Epoch : 0 [2750/23580 (12%)]	Loss: 2.191034	 Accuracy:41.488%
Epoch : 0 [3000/23580 (13%)]	Loss: 0.788812	 Accuracy:43.594%
Epoch : 0 [3250/23580 (14%)]	Loss: 0.788342	 Accuracy:45.499%
Epoch : 0 [3500/23580 (15%)]	Loss: 1.304036	 Accuracy:47.104%
Epoch : 0 [3750/23580 (16%)]	Loss: 0.403041	 Accuracy:48.868%
Epoch

In [0]:
torch.save(model_ft,"/content/drive/My Drive/ICNN_weights.pt")

In [0]:
torch.save(model_ft.state_dict(),"/content/drive/My Drive/ICNNst_weights.pt")

In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(icnn)    

9970890

In [0]:
#train fc layers of 5th microcnn
model_ft5 = ICNN.ICNN_Alexnet(ucnn=ucnn,batch_size=5,n_max=5)

for child in model_ft5.ucnn.children():
    child.conv1b.weight.requires_grad = False
    child.conv2.weight.requires_grad = False
    child.conv3.weight.requires_grad = False
    child.conv4.weight.requires_grad = False
    child.conv5.weight.requires_grad = False
    child.conv1b.bias.requires_grad = False
    child.conv2.bias.requires_grad = False
    child.conv3.bias.requires_grad = False
    child.conv4.bias.requires_grad = False
    child.conv5.bias.requires_grad = False
model_ft5.load_state_dict(torch.load("/content/drive/My Drive/ICNNst_weights.pt"))
model_ft5.eval()
model_ft5 = train_model_p1(model_ft5, criterion, optimizer_ft, mul_lr_scheduler,num_epochs=4)

In [0]:
torch.save(model_ft5.state_dict(),"/content/drive/My Drive/ICNNfinal_weights.pt")

In [0]:
#train fc layers of 4th microcnn
model_ft5 = ICNN.ICNN_Alexnet(ucnn=ucnn,batch_size=5,n_max=4)

for child in model_ft5.ucnn.children():
    child.conv1b.weight.requires_grad = False
    child.conv2.weight.requires_grad = False
    child.conv3.weight.requires_grad = False
    child.conv4.weight.requires_grad = False
    child.conv5.weight.requires_grad = False
    child.conv1b.bias.requires_grad = False
    child.conv2.bias.requires_grad = False
    child.conv3.bias.requires_grad = False
    child.conv4.bias.requires_grad = False
    child.conv5.bias.requires_grad = False
model_ft5.load_state_dict(torch.load("/content/drive/My Drive/ICNNfinal_weights.pt"))
model_ft5.eval()
model_ft5 = train_model_p1(model_ft5, criterion, optimizer_ft, mul_lr_scheduler,num_epochs=4)

In [0]:
torch.save(model_ft5.state_dict(),"/content/drive/My Drive/ICNNfinal_weights.pt")

In [0]:
#train fc layers of 3rd microcnn
model_ft5 = ICNN.ICNN_Alexnet(ucnn=ucnn,batch_size=5,n_max=3)

for child in model_ft5.ucnn.children():
    child.conv1b.weight.requires_grad = False
    child.conv2.weight.requires_grad = False
    child.conv3.weight.requires_grad = False
    child.conv4.weight.requires_grad = False
    child.conv5.weight.requires_grad = False
    child.conv1b.bias.requires_grad = False
    child.conv2.bias.requires_grad = False
    child.conv3.bias.requires_grad = False
    child.conv4.bias.requires_grad = False
    child.conv5.bias.requires_grad = False
model_ft5.load_state_dict(torch.load("/content/drive/My Drive/ICNNfinal_weights.pt"))
model_ft5.eval()
model_ft5 = train_model_p1(model_ft5, criterion, optimizer_ft, mul_lr_scheduler,num_epochs=4)

In [0]:
torch.save(model_ft5.state_dict(),"/content/drive/My Drive/ICNNfinal_weights.pt")

In [0]:
#train fc layers of 2nd microcnn
model_ft5 = ICNN.ICNN_Alexnet(ucnn=ucnn,batch_size=5,n_max=2)

for child in model_ft5.ucnn.children():
    child.conv1b.weight.requires_grad = False
    child.conv2.weight.requires_grad = False
    child.conv3.weight.requires_grad = False
    child.conv4.weight.requires_grad = False
    child.conv5.weight.requires_grad = False
    child.conv1b.bias.requires_grad = False
    child.conv2.bias.requires_grad = False
    child.conv3.bias.requires_grad = False
    child.conv4.bias.requires_grad = False
    child.conv5.bias.requires_grad = False
model_ft5.load_state_dict(torch.load("/content/drive/My Drive/ICNNfinal_weights.pt"))
model_ft5.eval()
model_ft5 = train_model_p1(model_ft5, criterion, optimizer_ft, mul_lr_scheduler,num_epochs=3)

In [0]:
torch.save(model_ft5.state_dict(),"/content/drive/My Drive/ICNNfinal_weights.pt")

In [0]:
#train fc layers of 1st microcnn
model_ft5 = ICNN.ICNN_Alexnet(ucnn=ucnn,batch_size=5,n_max=1)

for child in model_ft5.ucnn.children():
    child.conv1b.weight.requires_grad = False
    child.conv2.weight.requires_grad = False
    child.conv3.weight.requires_grad = False
    child.conv4.weight.requires_grad = False
    child.conv5.weight.requires_grad = False
    child.conv1b.bias.requires_grad = False
    child.conv2.bias.requires_grad = False
    child.conv3.bias.requires_grad = False
    child.conv4.bias.requires_grad = False
    child.conv5.bias.requires_grad = False
model_ft5.load_state_dict(torch.load("/content/drive/My Drive/ICNNfinal_weights.pt"))
model_ft5.eval()
model_ft5 = train_model_p1(model_ft5, criterion, optimizer_ft, mul_lr_scheduler,num_epochs=3)

In [0]:
torch.save(model_ft5.state_dict(),"/content/drive/My Drive/ICNNfinal_weights.pt")

In [0]:
from torch.autograd import Variable

In [0]:
def testing(model):
    correct = 0 
    for test_imgs, test_labels in dataset_test:
        #print(test_imgs.shape)
        test_imgs = Variable(test_imgs).float()
        output = model(test_imgs)
        predicted = torch.max(output,1)[1]
        correct += (predicted == test_labels).sum()
    print("Test accuracy:{:.3f}% ".format( float(correct) / (len(dataset_test)*dataset_test.batch_size)))

In [0]:
#testing the model on test set
testing(model_ft5)

Test accuracy:0.988% 
